In [1]:
import os
os.chdir("../")

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from category_encoders.ordinal import OrdinalEncoder
from category_encoders.cat_boost import CatBoostEncoder
from category_encoders.target_encoder import TargetEncoder
from scripts.data_clean import Keep_Features, Remove_NaN_Data
from scripts.utils import get_data_type
from scripts.data_transform import Tranform_Numerical_Data, Transform_Bool_Data, Transform_Categorical_Data

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Importação dos dados

In [3]:
df = pd.read_csv("data/pre_processed_data.csv")

In [4]:
df.head()

,id,fl_matriz,de_natureza_juridica,sg_uf,natureza_juridica_macro,de_ramo,setor,idade_empresa_anos,idade_emp_cat,fl_me,...,idade_minima_socios,qt_socios_st_regular,de_faixa_faturamento_estimado,de_faixa_faturamento_estimado_grupo,vl_faturamento_estimado_aux,vl_faturamento_estimado_grupo_aux,qt_filiais,emp_1,emp_2,emp_3
0,a6984c3ae395090e3bee8ad63c3758b110de096d5d8195...,True,SOCIEDADE EMPRESARIA LIMITADA,RN,ENTIDADES EMPRESARIAIS,INDUSTRIA DA CONSTRUCAO,CONSTRUÇÃO CIVIL,14.457534,10 a 15,False,...,41.0,2.0,"DE R$ 1.500.000,01 A R$ 4.800.000,00","DE R$ 1.500.000,01 A R$ 4.800.000,00",3132172.8,3132172.8,0,0,0,0
1,6178f41ade1365e44bc2c46654c2c8c0eaae27dcb476c4...,True,EMPRESARIO INDIVIDUAL,PI,OUTROS,SERVICOS DE ALOJAMENTO/ALIMENTACAO,SERVIÇO,1.463014,1 a 5,False,...,27.0,1.0,"DE R$ 81.000,01 A R$ 360.000,00","DE R$ 81.000,01 A R$ 360.000,00",210000.0,210000.0,0,0,0,0
2,4a7e5069a397f12fdd7fd57111d6dc5d3ba558958efc02...,True,EMPRESARIO INDIVIDUAL,AM,OUTROS,"TRANSPORTE, ARMAZENAGEM E CORREIO",SERVIÇO,7.093151,5 a 10,False,...,32.0,NaN,"ATE R$ 81.000,00","ATE R$ 81.000,00",50000.0,50000.0,0,0,0,0
3,3348900fe63216a439d2e5238c79ddd46ede454df7b9d8...,True,EMPRESARIO INDIVIDUAL,AM,OUTROS,SERVICOS DIVERSOS,SERVIÇO,6.512329,5 a 10,False,...,36.0,1.0,"DE R$ 81.000,01 A R$ 360.000,00","DE R$ 81.000,01 A R$ 360.000,00",210000.0,210000.0,0,0,0,0
4,1f9bcabc9d3173c1fe769899e4fac14b053037b953a1e4...,True,EMPRESARIO INDIVIDUAL,RN,OUTROS,"SERVICOS PROFISSIONAIS, TECNICOS E CIENTIFICOS",SERVIÇO,3.200000,1 a 5,False,...,NaN,NaN,"DE R$ 81.000,01 A R$ 360.000,00","DE R$ 81.000,01 A R$ 360.000,00",210000.0,210000.0,0,0,0,0


# Verificação dos valores nulos

In [5]:
df_na = pd.DataFrame(data = {"Features": 100*df.isna().sum()/df.shape[0]}).sort_values(by = "Features", 
                                                                       ascending = False)

In [6]:
df_na.head()

,Features
qt_socios_st_regular,33.510203
idade_media_socios,32.793133
idade_maxima_socios,32.793133
idade_minima_socios,32.793133
empsetorcensitariofaixarendapopulacao,31.111750


# Imputação dos valores faltantes

## Remoção de alguns valores nulos

A *feature* setor tem valores faltantes que impactam em várias outras variáveis. Devido a isso todas as linhas que tenham um valor nulo nessa *feature* são eliminadas.

In [7]:
df.dropna(subset = ["setor"], inplace = True)

Algumas *features* sem utilidade para previsão serão removidas como, **id** e **dt_situacao**.

In [8]:
emp_id = df["id"] #o ID será salvo para futuras consultas
df.drop(columns = ["id", "dt_situacao"], inplace = True)

In [9]:
df = df.reset_index(drop = True)

Os valores numéricos faltantes foram substituídos por **0**, os valores categóricos foram substituídos por **OUTROS** e os valores boolenos faltantes foram substituídos por **FALSE**.

In [10]:
CP = make_pipeline(Remove_NaN_Data(), Keep_Features("all"))
df = CP.fit_transform(df)

In [11]:
df.head()

,fl_matriz,de_natureza_juridica,sg_uf,natureza_juridica_macro,de_ramo,setor,idade_empresa_anos,idade_emp_cat,fl_me,fl_sa,...,idade_minima_socios,qt_socios_st_regular,de_faixa_faturamento_estimado,de_faixa_faturamento_estimado_grupo,vl_faturamento_estimado_aux,vl_faturamento_estimado_grupo_aux,qt_filiais,emp_1,emp_2,emp_3
0,True,SOCIEDADE EMPRESARIA LIMITADA,RN,ENTIDADES EMPRESARIAIS,INDUSTRIA DA CONSTRUCAO,CONSTRUÇÃO CIVIL,14.457534,10 a 15,False,False,...,41.0,2.0,"DE R$ 1.500.000,01 A R$ 4.800.000,00","DE R$ 1.500.000,01 A R$ 4.800.000,00",3132172.8,3132172.8,0,0,0,0
1,True,EMPRESARIO INDIVIDUAL,PI,OUTROS,SERVICOS DE ALOJAMENTO/ALIMENTACAO,SERVIÇO,1.463014,1 a 5,False,False,...,27.0,1.0,"DE R$ 81.000,01 A R$ 360.000,00","DE R$ 81.000,01 A R$ 360.000,00",210000.0,210000.0,0,0,0,0
2,True,EMPRESARIO INDIVIDUAL,AM,OUTROS,"TRANSPORTE, ARMAZENAGEM E CORREIO",SERVIÇO,7.093151,5 a 10,False,False,...,32.0,0.0,"ATE R$ 81.000,00","ATE R$ 81.000,00",50000.0,50000.0,0,0,0,0
3,True,EMPRESARIO INDIVIDUAL,AM,OUTROS,SERVICOS DIVERSOS,SERVIÇO,6.512329,5 a 10,False,False,...,36.0,1.0,"DE R$ 81.000,01 A R$ 360.000,00","DE R$ 81.000,01 A R$ 360.000,00",210000.0,210000.0,0,0,0,0
4,True,EMPRESARIO INDIVIDUAL,RN,OUTROS,"SERVICOS PROFISSIONAIS, TECNICOS E CIENTIFICOS",SERVIÇO,3.200000,1 a 5,False,False,...,0.0,0.0,"DE R$ 81.000,01 A R$ 360.000,00","DE R$ 81.000,01 A R$ 360.000,00",210000.0,210000.0,0,0,0,0


In [12]:
df_na = pd.DataFrame(data = {"Features": 100*df.isna().sum()/df.shape[0]}).sort_values(by = "Features", 
                                                                       ascending = False)

In [13]:
df_na.head()

,Features
fl_matriz,0.0
qt_socios_pj,0.0
de_saude_rescencia,0.0
nu_meses_rescencia,0.0
de_nivel_atividade,0.0


# TODO

## Feature creation

Nessa etapa são criadas novas *features*.

# Processamento dos dados

Nessa etapa os dados serão transformados. Algumas *features* serão geradas e serão aplicadas técnicas como *encoding* dos *labels* e padronização dos dados.

In [16]:
bool_data, object_data, numerical_data = get_data_type(df)

In [17]:
TRP = make_pipeline(Transform_Bool_Data(bool_data),
                    Transform_Categorical_Data(object_data),
                    Tranform_Numerical_Data(numerical_data),
                    Keep_Features("all"))

In [18]:
df_pr = TRP.fit_transform(df)
df_pr = pd.DataFrame(df_pr, columns = df.columns)
df_pr.head()

,fl_matriz,de_natureza_juridica,sg_uf,natureza_juridica_macro,de_ramo,setor,idade_empresa_anos,idade_emp_cat,fl_me,fl_sa,...,idade_minima_socios,qt_socios_st_regular,de_faixa_faturamento_estimado,de_faixa_faturamento_estimado_grupo,vl_faturamento_estimado_aux,vl_faturamento_estimado_grupo_aux,qt_filiais,emp_1,emp_2,emp_3
0,0.258667,1.845406,0.925304,-1.629903,-0.155488,-0.293631,0.485060,-0.669280,-0.05142,-0.136104,...,0.599322,0.484076,-1.486100,-1.394300,0.078853,-0.045536,-0.061444,-0.034742,-0.035085,-0.023999
1,0.258667,-0.440166,0.217005,0.639601,0.809181,1.111097,-0.882350,-1.214972,-0.05142,-0.136104,...,-0.039441,0.031070,0.631962,0.690613,-0.018143,-0.045959,-0.061444,-0.034742,-0.035085,-0.023999
2,0.258667,-0.440166,-1.199593,0.639601,1.988220,1.111097,-0.289892,0.422105,-0.05142,-0.136104,...,0.188689,-0.421936,-1.750858,-1.654914,-0.023454,-0.045982,-0.061444,-0.034742,-0.035085,-0.023999
3,0.258667,-0.440166,-1.199593,0.639601,1.237922,1.111097,-0.351012,0.422105,-0.05142,-0.136104,...,0.371192,0.031070,0.631962,0.690613,-0.018143,-0.045959,-0.061444,-0.034742,-0.035085,-0.023999
4,0.258667,-0.440166,0.925304,0.639601,1.452293,1.111097,-0.699568,-1.214972,-0.05142,-0.136104,...,-1.271340,-0.421936,0.631962,0.690613,-0.018143,-0.045959,-0.061444,-0.034742,-0.035085,-0.023999


# Feature selection

Nessa etapa serão selecionadas *features* para utilização no modelo final.

Primeiramente serão removidas *features* com apenas um valor.

In [20]:
def get_unique_features(dataframe):
    drop_features = []
    for col in dataframe.columns:
        if dataframe[col].nunique() == 1:
            drop_features.append(col)
    return drop_features

In [23]:
drop_features = get_unique_features(df_pr)
df_pr.drop(columns = drop_features, inplace = True)

In [24]:
df_pr = df_pr.drop(columns = ["emp_1", "emp_2", "emp_3"])

In [25]:
df_pr["id"] = emp_id

In [28]:
df_pr.head()

,fl_matriz,de_natureza_juridica,sg_uf,natureza_juridica_macro,de_ramo,setor,idade_empresa_anos,idade_emp_cat,fl_me,fl_sa,...,idade_media_socios,idade_maxima_socios,idade_minima_socios,qt_socios_st_regular,de_faixa_faturamento_estimado,de_faixa_faturamento_estimado_grupo,vl_faturamento_estimado_aux,vl_faturamento_estimado_grupo_aux,qt_filiais,id
0,0.258667,1.845406,0.925304,-1.629903,-0.155488,-0.293631,0.485060,-0.669280,-0.05142,-0.136104,...,0.671323,0.722216,0.599322,0.484076,-1.486100,-1.394300,0.078853,-0.045536,-0.061444,a6984c3ae395090e3bee8ad63c3758b110de096d5d8195...
1,0.258667,-0.440166,0.217005,0.639601,0.809181,1.111097,-0.882350,-1.214972,-0.05142,-0.136104,...,-0.081970,-0.121498,-0.039441,0.031070,0.631962,0.690613,-0.018143,-0.045959,-0.061444,6178f41ade1365e44bc2c46654c2c8c0eaae27dcb476c4...
2,0.258667,-0.440166,-1.199593,0.639601,1.988220,1.111097,-0.289892,0.422105,-0.05142,-0.136104,...,0.139587,0.089430,0.188689,-0.421936,-1.750858,-1.654914,-0.023454,-0.045982,-0.061444,4a7e5069a397f12fdd7fd57111d6dc5d3ba558958efc02...
3,0.258667,-0.440166,-1.199593,0.639601,1.237922,1.111097,-0.351012,0.422105,-0.05142,-0.136104,...,0.316832,0.258173,0.371192,0.031070,0.631962,0.690613,-0.018143,-0.045959,-0.061444,3348900fe63216a439d2e5238c79ddd46ede454df7b9d8...
4,0.258667,-0.440166,0.925304,0.639601,1.452293,1.111097,-0.699568,-1.214972,-0.05142,-0.136104,...,-1.278376,-1.260513,-1.271340,-0.421936,0.631962,0.690613,-0.018143,-0.045959,-0.061444,1f9bcabc9d3173c1fe769899e4fac14b053037b953a1e4...


In [26]:
df_pr.to_csv("data/data_to_use.csv", index = False)